In [2]:
import sys
import os
from tqdm import tqdm
import pandas as pd
sys.path.append("..")
#sys.path.append("/ihome/minsanally/tih85/EphysAnalysis/")
sys.path.append("C:\\Users\insan\\Desktop\\EphysAnalysis")
import InsanallyLabEphysTools as ilep

from collections import OrderedDict
from itertools import product
import multiprocessing as mp
import numpy as np
import pickle




results_params = []
results_async = []
def log_result(result, final_list=results_async):
    final_list.append(result)

def log_error(x, final_list=results_async):
    final_list.append({})






if __name__ == "__main__":
    full_dir='C:\\Users\insan\\Desktop\\M2 decoding test'
    print("Starting experiment_run_stimulus_decoding_25s_M2.py")

    CACHE_DIRECTORY = 'D:\\Analysis_Cache'
    OUTPUT_DIRECTORY = 'C:\\Users\insan\\Desktop\\M2 decoding test\\Output'
    REPETITIONS = 10
    CATEGORIES = 'stimulus'

    pool = mp.Pool(mp.cpu_count())

    ####################################################################################################################################
    n_reps = range(REPETITIONS)

    EnumSession = []
    EnumClust = []
    sessions = os.listdir(CACHE_DIRECTORY)
    for session in sessions:
        sessionfile = ilep.loadSessionCached(CACHE_DIRECTORY,session)

        if sessionfile.meta.task == 'passive no behavior':
            continue
        if sessionfile.meta.task in ['tuning nonreversal','tuning switch','tuning reversal']:
            continue
        if sessionfile.meta.region != 'M2':
            continue
        
        for clust in sessionfile.clusters.good:
            EnumSession.append(session)
            EnumClust.append(clust)

    ####################################################################################################################################

    EnumSession = EnumSession[0:20]
    EnumClust = EnumClust[0:20]
    print(EnumSession)
    print(EnumClust)
    
    try:
        with open('C:\\Users\\insan\\Desktop\\trialsToUsePerDay', 'rb') as f:
            trialsPerDayLoaded = pickle.load(f)
    except Exception as e:
        print(e)
        raise e

    ####################################################################################################################################

    progress_bar = tqdm(zip(EnumSession,EnumClust), desc=f"Calculating {CATEGORIES} decoding")
    for session,clust in progress_bar:


        results = {}
        results['n_rep'] = REPETITIONS
        results['categories'] = CATEGORIES
        results['session'] = session
        results['clust'] = clust
        try:
            progress_bar.write(f"{session} cluster {clust} is present.")
            
            #Need to create interval
            trainInterval = ilep.TrialInterval(-0.2*30000,2.5*30000,False,False)
            testInterval = ilep.TrialInterval(0,2.5*30000,False,False)
            temp = pool.apply_async(ilep.calculateDecodingForSingleNeuron,(session,clust,trialsPerDayLoaded,CACHE_DIRECTORY,OUTPUT_DIRECTORY,trainInterval,testInterval,REPETITIONS,CATEGORIES))
            progress_bar.write(f"{session} cluster {clust}: {temp}")
            results_async.append(temp)
            results_params.append(results)

        except Exception as e:
            progress_bar.write(f"{session},{clust}: {e}")
            continue

    # Closing the worker pool
    pool.close()
    pool.join()

    results_2 = []
    for r in results_async:
        try:
            results_2.append(r.get())
        except Exception as e:
            results_2.append({})
    
    progress_bar.write(f"results_2: {results_2}")
    progress_bar.write(f"results_async: {results_async}")

    # Combining results
    total_results = [{**d1, **d2} for d1, d2 in zip(results_params, results_2)]

    # Saving as CSV
    results_df = pd.DataFrame(total_results)

    variables=['session','clust']
    results_df = results_df.sort_values(by=variables).reset_index(drop=True)
    results_df.to_csv(os.path.join(full_dir, "stimdecoding_25s_M2.csv"))

Starting experiment_run_stimulus_decoding_25s_M2.py
['BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle', 'BS_108_1_M2.pickle']
[19, 29, 45, 61, 64, 68, 69, 70, 71, 92, 95, 96, 120, 123, 128, 141, 142, 148, 169, 175]


BS_108_1_M2.pickle cluster 19 is present.
BS_108_1_M2.pickle cluster 19: <multiprocessing.pool.ApplyResult object at 0x000002698524A3D0>
BS_108_1_M2.pickle cluster 29 is present.
BS_108_1_M2.pickle cluster 29: <multiprocessing.pool.ApplyResult object at 0x000002698524A430>
BS_108_1_M2.pickle cluster 45 is present.
BS_108_1_M2.pickle cluster 45: <multiprocessing.pool.ApplyResult object at 0x000002698597B1C0>
BS_108_1_M2.pickle cluster 61 is present.
BS_108_1_M2.pickle cluster 61: <multiprocessing.pool.ApplyResult object at 0x00000269F1409820>
BS_108_1_M2.pickle cluster 64 is present.
BS_108_1_M2.pickle cluster 64: <multiprocessing.pool.ApplyResult object at 0x0000026985C08E50>
BS_108_1_M2.pickle cluster 68 is present.
BS_108_1_M2.pickle cluster 68: <multiprocessing.pool.ApplyResult object at 0x0000026987900070>
BS_108_1_M2.pickle cluster 69 is present.
BS_108_1_M2.pickle cluster 69: <multiprocessing.pool.ApplyResult object at 0x0000026987900400>
BS_108_1_M2.pickle cluster 70 is present.

Calculating stimulus decoding: 20it [00:00, 86.81it/s]


BS_108_1_M2.pickle cluster 96: <multiprocessing.pool.ApplyResult object at 0x00000269F14814C0>
BS_108_1_M2.pickle cluster 120 is present.
BS_108_1_M2.pickle cluster 120: <multiprocessing.pool.ApplyResult object at 0x00000269F14813D0>
BS_108_1_M2.pickle cluster 123 is present.
BS_108_1_M2.pickle cluster 123: <multiprocessing.pool.ApplyResult object at 0x00000269F1481670>
BS_108_1_M2.pickle cluster 128 is present.
BS_108_1_M2.pickle cluster 128: <multiprocessing.pool.ApplyResult object at 0x00000269F14817F0>
BS_108_1_M2.pickle cluster 141 is present.
BS_108_1_M2.pickle cluster 141: <multiprocessing.pool.ApplyResult object at 0x00000269F1481AF0>
BS_108_1_M2.pickle cluster 142 is present.
BS_108_1_M2.pickle cluster 142: <multiprocessing.pool.ApplyResult object at 0x00000269F1481CD0>
BS_108_1_M2.pickle cluster 148 is present.
BS_108_1_M2.pickle cluster 148: <multiprocessing.pool.ApplyResult object at 0x0000026988E96880>
BS_108_1_M2.pickle cluster 169 is present.
BS_108_1_M2.pickle cluster 1